In [1]:
from dataclasses import dataclass

@dataclass(unsafe_hash=True)
class Brick:
    number: int
    x1: int
    y1: int
    z1: int
    x2: int
    y2: int
    z2: int
    
    @property
    def coords(self):
        return self.x1, self.y1, self.z1, self.x2, self.y2, self.z2

    @property
    def cubes(self):
        return get_cubes(*self.coords)

In [2]:
from itertools import product
from functools import cache

@cache
def get_cubes(x1, y1, z1, x2, y2, z2):
    xr = range(min(x1, x2), max(x1, x2) + 1)
    yr = range(min(y1, y2), max(y1, y2) + 1)
    zr = range(min(z1, z2), max(z1, z2) + 1)
    return set(product(xr, yr, zr))

def get_bricks(brick, z=0):
    cubes = get_cubes(brick.x1, brick.y1, brick.z1 + z, brick.x2, brick.y2, brick.z2 + z)
    return set(other for other in bricks if other != brick and cubes & other.cubes)

In [3]:
import re

bricks = []
with open("Day22.txt") as file:
    for number, line in enumerate(file, start=1):
        brick = Brick(number, *map(int, re.findall(r"\d+", line)))
        bricks.append(brick)

In [4]:
all_cubes = set()
for brick in sorted(bricks, key=lambda b: max(b.z1, b.z2)):
    cubes = brick.cubes
    all_other_cubes = all_cubes - cubes
    z1, z2 = brick.z1, brick.z2
    while z1 > 0 and z2 > 0 and not cubes & all_other_cubes:
        brick.z1, brick.z2 = z1, z2
        z1, z2 = z1 - 1, z2 - 1
        cubes = get_cubes(brick.x1, brick.y1, z1, brick.x2, brick.y2, z2)
    all_cubes.update(brick.cubes)

In [5]:
bricks_above, bricks_below = {}, {}
for brick in bricks:
    bricks_above[brick] = get_bricks(brick, +1)
    bricks_below[brick] = get_bricks(brick, -1)

In [6]:
safe_bricks = set()
for brick in bricks:
    if all(len(bricks_below[above]) > 1 for above in bricks_above[brick]):
        safe_bricks.add(brick)
len(safe_bricks)

465

In [7]:
def falling_bricks(brick, falling=None):
    falling = falling or set()
    falling.add(brick)
    for above in bricks_above[brick]:
        if not bricks_below[above] - falling:
            falling.update(falling_bricks(above, falling))
    return falling

sum(len(falling_bricks(brick)) - 1 for brick in bricks if brick not in safe_bricks)

79042